In [1]:
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.fem.petsc import LinearProblem, NonlinearProblem
import dolfinx
import meshio
import gmsh
import time


In [2]:
mesh_path = "../meshes/mesh_square.msh"
output_path = "../output/interactive/interactive_fenics.vtu"

In [3]:
start_time = time.time()

gmsh.initialize()
gmsh.clear()
gmsh.model.add("loaded_mesh")

gmsh.open(mesh_path)

# gmsh.finalize()

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Reading '../meshes/mesh_square.msh'...
Info    : 2017 nodes
Info    : 4032 elements
Info    : Done reading '../meshes/mesh_square.msh'


Explanation how to import mesh from GMsh:
https://jsdokken.com/FEniCS-workshop/src/external_mesh.html

In [4]:
# WARNING!!! It's important to specify that the mesh is 2D, otherwise it will perceive it as 3D
msh_data = dolfinx.io.gmsh.model_to_mesh(gmsh.model, MPI.COMM_SELF, 0, gdim=2)
msh = msh_data.mesh
cell_marker = msh_data.cell_tags
facet_marker = msh_data.facet_tags

# print(facet_marker.values)
# print(facet_marker.indices)
print(np.unique(facet_marker.values))

gmsh.finalize()

[1 2 3 4]


In [5]:
# boundaries = [
#     (3, lambda x: np.isclose(x[1], 0)),
#     (4, lambda x: np.isclose(x[1], 1)),
# ]

# facet_indices, facet_markers = [], []
# fdim = msh.topology.dim - 1
# for marker, locator in boundaries:
#     facets = mesh.locate_entities(msh, fdim, locator)
#     facet_indices.append(facets)
#     facet_markers.append(np.full_like(facets, marker))
# facet_indices = np.hstack(facet_indices).astype(np.int32)
# facet_markers = np.hstack(facet_markers).astype(np.int32)
# sorted_facets = np.argsort(facet_indices)
# facet_tag = mesh.meshtags(
#     msh, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets]
# )


In [6]:
V = fem.functionspace(msh, ("Lagrange", 1))

In [7]:
# facets = mesh.locate_entities_boundary(
#     msh,
#     dim=(msh.topology.dim - 1),
#     marker=lambda x: np.isclose(x[0], 0.0) | np.isclose(x[0], 1.0),
# )

left_tag = 1
facets_left = facet_marker.find(left_tag)
right_tag = 2
facets_right = facet_marker.find(right_tag)
top_tag = 3
# facets_top = facet_marker.find(top_tag)
bottom_tag = 4
# facets_bottom = facet_marker.find(bottom_tag)


dofs_left = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets_left)
dofs_right = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets_right)
# dofs_top = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets_top)
# dofs_bottom = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets_bottom)

bc_left = fem.dirichletbc(value=ScalarType(0), dofs=dofs_left, V=V)
bc_right = fem.dirichletbc(value=ScalarType(0), dofs=dofs_right, V=V)


In [8]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

uh = fem.Function(V)

x = ufl.SpatialCoordinate(msh)
# f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
g = 10 * ufl.sin(6 * x[0])
# a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
a = ufl.inner(ufl.grad(uh), ufl.grad(v)) * ufl.dx
# L = ufl.inner(f, v) * ufl.dx + ufl.inner(g, v) * ufl.ds

ds = ufl.Measure("ds", domain=msh, subdomain_data=facet_marker)

L = ufl.inner(g, v) * ds(top_tag) + ufl.inner(g, v) * ds(bottom_tag)

F = a - L

In [9]:
petsc_options = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "none",
    "snes_atol": 1e-6,
    "snes_rtol": 1e-6,
    "snes_monitor": None,
    "ksp_error_if_not_converged": True,
    "ksp_type": "cg",
    "ksp_rtol": 1e-7,
    "ksp_monitor": None,
    "pc_type": "hypre",
    "pc_hypre_type": "boomeramg",
    "pc_hypre_boomeramg_max_iter": 1,
    "pc_hypre_boomeramg_cycle_type": "v",
}

problem = NonlinearProblem(
    F,
    uh,
    bcs=[bc_left, bc_right],
    petsc_options=petsc_options,
    petsc_options_prefix="nonlinpoisson",
)

In [10]:
problem.solve()
converged = problem.solver.getConvergedReason()
num_iter = problem.solver.getIterationNumber()
assert converged > 0, f"Solver did not converge, got {converged}."
print(
    f"Solver converged after {num_iter} iterations with converged reason {converged}."
)

  0 SNES Function norm 1.611307087271e+00
    Residual norms for nonlinpoisson solve.
    0 KSP Residual norm 1.998089191318e+01
    1 KSP Residual norm 5.839478650028e-01
    2 KSP Residual norm 1.987608584242e-02
    3 KSP Residual norm 9.276318319550e-04
    4 KSP Residual norm 4.487463142198e-05
    5 KSP Residual norm 1.298041503636e-06
  1 SNES Function norm 3.454392528732e-07
Solver converged after 1 iterations with converged reason 2.


In [11]:
# with io.XDMFFile(msh.comm, "output/square/poisson_sqaure_fenics.xdmf", "w") as file:
#     file.write_mesh(msh)
#     file.write_function(uh)

# with io.VTKFile(msh.comm, "output/square/poisson_sqaure_fenics.pvd", "w") as file:
#     file.write_mesh(msh)
#     file.write_function(uh, t=0.0)

Evaluating function at points: https://jsdokken.com/FEniCS-workshop/src/deep_dive/expressions.html#interpolation-on-a-subset-of-cells

In [12]:
# Evaluating solution function uh at mesh points 

# import pyvista as pv

# res = pv.read(mesh_path)
# points = res.points

# bb_tree = dolfinx.geometry.bb_tree(msh, msh.topology.dim, padding=1e-10)
# potential_colliding_cells = dolfinx.geometry.compute_collisions_points(bb_tree, points)

# colliding_cells = dolfinx.geometry.compute_colliding_cells(msh, potential_colliding_cells, points)

# points_on_proc = []
# cells = []
# for i, point in enumerate(points):
#     if len(colliding_cells.links(i)) > 0:
#         points_on_proc.append(point)
#         cells.append(colliding_cells.links(i)[0])
# points_on_proc = np.array(points_on_proc, dtype=np.float64).reshape(-1, 3)
# cells = np.array(cells, dtype=np.int32)
# sol = uh.eval(points_on_proc, cells)

# res["sol"] = sol

# res.save(output_path)

In [13]:
# Extracting solution vector from solution function uh 
# and re-arranging dofs to match mesh dof order to match what we did in NGSolve example
import numpy as np
from scipy.spatial import KDTree
import pyvista as pv

res = pv.read(mesh_path)
points = res.points

sol_unordered = uh.x.array
points_unordered = V.tabulate_dof_coordinates()

tree2 = KDTree(points_unordered)

_, indices = tree2.query(points)

sol = sol_unordered[indices]


In [14]:
res["sol"] = sol

res.save(output_path)